In [ ]:
%cd ..

In [ ]:
import argparse, os, sys, glob
import torch
import numpy as np
from omegaconf import OmegaConf
from PIL import Image
from tqdm import tqdm, trange
from itertools import islice
from einops import rearrange
from torchvision.utils import make_grid, save_image
import time
from pytorch_lightning import seed_everything
from torch import autocast
from contextlib import contextmanager, nullcontext

from ldm.util import instantiate_from_config
from ldm.models.diffusion.ddim import DDIMSampler
from ldm.models.diffusion.plms import PLMSSampler

import warnings
warnings.filterwarnings('ignore')

In [ ]:
def chunk(it, size):
    it = iter(it)
    return iter(lambda: tuple(islice(it, size)), ())


def load_model_from_config(config, ckpt, verbose=False):
    print(f"Loading model from {ckpt}")
    pl_sd = torch.load(ckpt, map_location="cpu")
    if "global_step" in pl_sd:
        print(f"Global Step: {pl_sd['global_step']}")
    sd = pl_sd["state_dict"]
    model = instantiate_from_config(config.model)
    m, u = model.load_state_dict(sd, strict=False)
    if len(m) > 0 and verbose:
        print("missing keys:")
        print(m)
    if len(u) > 0 and verbose:
        print("unexpected keys:")
        print(u)

    model.cuda()
    model.eval()
    return model

In [ ]:
def parse_args():
    parser = argparse.ArgumentParser()

    parser.add_argument(
        "--prompt",
        type=str,
        nargs="?",
        default="a painting of a virus monster playing guitar",
        help="the prompt to render"
    )
    parser.add_argument(
        "--outdir",
        type=str,
        nargs="?",
        help="dir to write results to",
        default="outputs/txt2img-samples"
    )
    parser.add_argument(
        "--skip_grid",
        action='store_true',
        help="do not save a grid, only individual samples. Helpful when evaluating lots of samples",
    )
    parser.add_argument(
        "--skip_save",
        action='store_true',
        help="do not save individual samples. For speed measurements.",
    )
    parser.add_argument(
        "--ddim_steps",
        type=int,
        default=100,
        help="number of ddim sampling steps",
    )
    parser.add_argument(
        "--plms",
        action='store_true',
        help="use plms sampling",
    )
    parser.add_argument(
        "--laion400m",
        action='store_true',
        help="uses the LAION400M model",
    )
    parser.add_argument(
        "--fixed_code",
        action='store_true',
        help="if enabled, uses the same starting code across samples ",
    )
    parser.add_argument(
        "--ddim_eta",
        type=float,
        default=0.0,
        help="ddim eta (eta=0.0 corresponds to deterministic sampling",
    )
    parser.add_argument(
        "--n_iter",
        type=int,
        default=1,
        help="sample this often",
    )
    parser.add_argument(
        "--H",
        type=int,
        default=512,
        help="image height, in pixel space",
    )
    parser.add_argument(
        "--W",
        type=int,
        default=512,
        help="image width, in pixel space",
    )
    parser.add_argument(
        "--C",
        type=int,
        default=4,
        help="latent channels",
    )
    parser.add_argument(
        "--f",
        type=int,
        default=8,
        help="downsampling factor",
    )
    parser.add_argument(
        "--n_samples",
        type=int,
        default=2,
        help="how many samples to produce for each given prompt. A.k.a. batch size",
    )
    parser.add_argument(
        "--n_rows",
        type=int,
        default=2,
        help="rows in the grid (default: n_samples)",
    )
    parser.add_argument(
        "--scale",
        type=float,
        default=7.5,
        help="unconditional guidance scale: eps = eps(x, empty) + scale * (eps(x, cond) - eps(x, empty))",
    )
    parser.add_argument(
        "--from-file",
        type=str,
        help="if specified, load prompts from this file",
    )
    parser.add_argument(
        "--config",
        type=str,
        default="configs/stable-diffusion/v1-inference.yaml",
        help="path to config which constructs model",
    )
    parser.add_argument(
        "--ckpt",
        type=str,
        default="models/ldm/stable-diffusion-v4/sd-v1-4-full-ema.ckpt",
        help="path to checkpoint of model",
    )   
    
    parser.add_argument(
        "--sin_config",
        type=str,
        default="configs/stable-diffusion/v1-inference.yaml",
        nargs='+'
    )
    parser.add_argument(
        "--sin_ckpt",
        type=str,
        nargs='+'
    )
     
    parser.add_argument(
        "--seed",
        type=int,
        default=42,
        help="the seed (for reproducible sampling)",
    )
    parser.add_argument(
        "--precision",
        type=str,
        help="evaluate at this precision",
        choices=["full", "autocast"],
        default="full"
    )
    parser.add_argument("--single_guidance", action="store_true")
    parser.add_argument("--range_t_max", type=int, default=400, nargs='+')
    parser.add_argument("--range_t_min", type=int, default=1, nargs='+')
    parser.add_argument("--beta", type=float, default=0.5, nargs='+')

    parser.add_argument(
        "--embedding_path", 
        type=str, 
        help="Path to a pre-trained embedding manager checkpoint")
    
    return parser.parse_args(args=[])

In [ ]:
opt = parse_args()
opt.single_guidance=True
seed_everything(opt.seed)

LOG_DIR="./logs/FarFocus"
opt.sin_ckpt=[ LOG_DIR+"_Train/checkpoints/last.ckpt", LOG_DIR+"_Test/checkpoints/last.ckpt" ]
opt.sin_config = ['./configs/stable-diffusion/v1-inference.yaml', './configs/stable-diffusion/v1-inference.yaml']
opt.ckpt = './models/pretrained.ckpt'

config = OmegaConf.load(f"{opt.config}")
model = load_model_from_config(config, f"{opt.ckpt}")
#model.embedding_manager.load(opt.embedding_path)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model = model.to(device)

guidance_configs = opt.sin_config
guidance_ckpts = opt.sin_ckpt

assert len(guidance_configs) == len(guidance_ckpts)

num_guidance = len(guidance_configs)
guidance_models = [load_model_from_config(OmegaConf.load(config), ckpt).to(device) for config, ckpt in zip(guidance_configs, guidance_ckpts)]

from ldm.models.diffusion.guidance_ddim import DDIMMultiSampler
sampler = DDIMMultiSampler(model, guidance_models)

In [ ]:
def inference(opt, beta, tmax, tmin, ddim_eta, ddim_steps, scale):

    opt.beta = beta
    opt.range_t_max = tmax
    opt.range_t_min = tmin
    opt.ddim_eta = ddim_eta
    opt.scale = scale
    opt.ddim_steps = ddim_steps

    guidance_betas = opt.beta if isinstance(opt.beta, list) else [opt.beta]*num_guidance
    guidance_range_t_max = opt.range_t_max if isinstance(opt.range_t_max, list) else [opt.range_t_max]*num_guidance
    guidance_range_t_min = opt.range_t_min if isinstance(opt.range_t_min, list) else [opt.range_t_min]*num_guidance
    assert len(guidance_betas) == num_guidance and len(guidance_range_t_max) == num_guidance and len(guidance_range_t_min) == num_guidance

    for sin_model, beta, t_max, t_min in zip(sampler.guide_model_list, guidance_betas, guidance_range_t_max, guidance_range_t_min):
        sin_model.extra_config = {"beta": beta, "range_t_max": t_max, "range_t_min": t_min}

    prompt=opt.prompt

    start_code = None
    if opt.fixed_code:
        start_code = torch.randn([opt.n_samples, opt.C, opt.H // opt.f, opt.W // opt.f], device=device)

    precision_scope = autocast if opt.precision=="autocast" else nullcontext

    batch_size = opt.n_samples

    data = [batch_size * [prompt]]

    with torch.no_grad():
        with precision_scope("cuda"):
            with model.ema_scope():
                for sin_model in guidance_models:
                    sin_model.ema_scope()
                tic = time.time()
                all_samples = list()
                for n in range(opt.n_iter):
                    for prompts in data:
                        uc = None
                        if opt.scale != 1.0:
                            uc = model.get_learned_conditioning(batch_size * [""])
                            uc_sin_list = [sin_model.get_learned_conditioning(batch_size * [""]) for sin_model in guidance_models]
                        if isinstance(prompts, tuple):
                            prompts = list(prompts)

                        if opt.single_guidance:
                            b = len(prompts)
                            prompt = prompts[0]
                            prompts = [prompt.split('[SEP]')[0].strip()] * b

                            prompts_single = [[p.strip()] * b for p in prompt.split('[SEP]')[1:]]

                        c = model.get_learned_conditioning(prompts)
                        c_sin_list = [sin_model.get_learned_conditioning(p) for sin_model, p in zip(guidance_models, prompts_single)]

                        shape = [opt.C, opt.H // opt.f, opt.W // opt.f]
                        samples_ddim, _ = sampler.sample(S=opt.ddim_steps,
                                                        conditioning=c,
                                                        conditioning_single_list=c_sin_list,
                                                        batch_size=opt.n_samples,
                                                        shape=shape,
                                                        verbose=False,
                                                        unconditional_guidance_scale=opt.scale,
                                                        unconditional_conditioning=uc,
                                                        unconditional_conditioning_single_list=uc_sin_list,
                                                        eta=opt.ddim_eta,
                                                        x_T=start_code)

                        x_samples_ddim = model.decode_first_stage(samples_ddim)
                        x_samples_ddim = torch.clamp((x_samples_ddim + 1.0) / 2.0, min=0.0, max=1.0)

                        if not opt.skip_grid:
                            all_samples.append(x_samples_ddim)

                if not opt.skip_grid:
                    grid = torch.stack(all_samples, 0)
                    grid = rearrange(grid, 'n b c h w -> (n b) c h w')
                    grid = make_grid(grid, nrow=opt.n_rows)
                    grid = 255. * rearrange(grid, 'c h w -> h w c').cpu().numpy()
                    img=Image.fromarray(grid.astype(np.uint8))
    return img

In [ ]:
opt.prompt="A photo of a dog sitting in the library[SEP]"+\
           "A far foucus camera photo of a dog sitting in the library[SEP]"+\
           "A photo of a dog sitting in the library"
opt.n_iter=1

In [ ]:
sampler.guide_model_list = guidance_models
sampler.model = model
# Results of method 2
inference(opt=opt, beta=[0.2, 0.7], tmax=[1000,1000], tmin=[400,800], ddim_eta=0.0, ddim_steps=100, scale=15.0)

In [ ]:
sampler.guide_model_list = [guidance_models[1], guidance_models[1]]
opt.n_samples = 4
# Results of SINE
inference(opt=opt, beta=[0.2, 0.7], tmax=[1000,1000], tmin=[400,800], ddim_eta=0.0, ddim_steps=100, scale=15.0)